# A priority queue using AA Trees

An AA tree in computer science is a form of balanced tree used for storing and retrieving ordered data efficiently. AA trees are named for Arne Andersson, their inventor.

AA trees are a variation of the red-black tree, a form of binary search tree which supports efficient addition and deletion of entries. Unlike red-black trees, red nodes on an AA tree can only be added as a right subchild. In other words, no red node can be a left sub-child. This results in the simulation of a 2-3 tree instead of a 2-3-4 tree, which greatly simplifies the maintenance operations.

https://en.wikipedia.org/wiki/AA_tree

## Let's start with an implementation

https://www.nayuki.io/page/aa-tree-set

In [1]:
#
# AA tree set (Python)
#
# Copyright (c) 2018 Project Nayuki. (MIT License)
# https://www.nayuki.io/page/aa-tree-set
#
# Permission is hereby granted, free of charge, to any person obtaining a copy of
# this software and associated documentation files (the "Software"), to deal in
# the Software without restriction, including without limitation the rights to
# use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
# the Software, and to permit persons to whom the Software is furnished to do so,
# subject to the following conditions:
# - The above copyright notice and this permission notice shall be included in
#   all copies or substantial portions of the Software.
# - The Software is provided "as is", without warranty of any kind, express or
#   implied, including but not limited to the warranties of merchantability,
#   fitness for a particular purpose and noninfringement. In no event shall the
#   authors or copyright holders be liable for any claim, damages or other
#   liability, whether in an action of contract, tort or otherwise, arising from,
#   out of or in connection with the Software or the use or other dealings in the
#   Software.
#


class AaTreeSet(object):

    def __init__(self, coll=None):
        self.clear()
        if coll is not None:
            for val in coll:
                self.add(val)


    def clear(self):
        self.root = AaTreeSet.Node.EMPTY_LEAF
        self.size = 0


    def __len__(self):
        return self.size


    def __contains__(self, val):
        node = self.root
        while node is not AaTreeSet.Node.EMPTY_LEAF:
            if val < node.value:
                node = node.left
            elif val > node.value:
                node = node.right
            else:
                return True
        return False


    def add(self, val):
        if val in self:
            return
        self.root = self.root.add(val)
        self.size += 1


    def remove(self, val):
        self.root, found = self.root.remove(val)
        if not found:
            raise KeyError(str(val))
        self.size -= 1


    def discard(self, val):
        self.root, found = self.root.remove(val)
        if found:
            self.size -= 1


    # Note: Not fail-fast on concurrent modification.
    def __iter__(self):
        stack = []
        node = self.root
        while True:
            while node is not AaTreeSet.Node.EMPTY_LEAF:
                stack.append(node)
                node = node.left
            if len(stack) == 0:
                break
            node = stack.pop()
            yield node.value
            node = node.right


    class Node(object):

        def __init__(self, val=None):
            self.value = val
            if val is None:  # For the singleton empty leaf node
                self.level = 0
            else:  # Normal non-leaf nodes
                self.level = 1
                self.left  = AaTreeSet.Node.EMPTY_LEAF
                self.right = AaTreeSet.Node.EMPTY_LEAF


        def add(self, val):
            if self is AaTreeSet.Node.EMPTY_LEAF:
                return AaTreeSet.Node(val)
            if val < self.value:
                self.left = self.left.add(val)
            elif val > self.value:
                self.right = self.right.add(val)
            else:
                raise ValueError("Value already in tree")
            return self._skew()._split()  # Rebalance this node


        def remove(self, val):
            EMPTY = AaTreeSet.Node.EMPTY_LEAF
            if self is EMPTY:
                return (EMPTY, False)

            if val < self.value:
                self.left, found = self.left.remove(val)
            elif val > self.value:
                self.right, found = self.right.remove(val)
            else:  # Remove value at this node
                found = True
                if self.left is not EMPTY:
                    # Find predecessor node
                    temp = self.left
                    while temp.right is not EMPTY:
                        temp = temp.right
                    self.value = temp.value  # Replace value with predecessor
                    self.left, fnd = self.left.remove(self.value)  # Remove predecessor node
                    assert fnd
                elif self.right is not EMPTY:
                    # Find successor node
                    temp = self.right
                    while temp.left is not EMPTY:
                        temp = temp.left
                    self.value = temp.value  # Replace value with successor
                    self.right, fnd = self.right.remove(self.value)  # Remove successor node
                    assert fnd
                else:
                    assert self.level == 1
                    return (EMPTY, True)

            # Rebalance this node if a child was lowered
            if not found or self.level == min(self.left.level, self.right.level) + 1:
                return (self, found)
            if self.right.level == self.level:
                self.right.level -= 1
            self.level -=1
            result = self._skew()
            result.right = result.right._skew()
            if result.right.right is not EMPTY:
                result.right.right = result.right.right._skew()
            result = result._split()
            result.right = result.right._split()
            return (result, True)


        def _skew(self):
            assert self is not AaTreeSet.Node.EMPTY_LEAF
            if self.left.level < self.level:
                return self
            result = self.left
            self.left = result.right
            result.right = self
            return result


        def _split(self):
            assert self is not AaTreeSet.Node.EMPTY_LEAF
            # Must short-circuit because if right.level < self.level, then right.right might not exist
            if self.right.level < self.level or self.right.right.level < self.level:
                return self
            result = self.right
            self.right = result.left
            result.left = self
            result.level += 1
            return result


# Static initializer. A bit of a hack, but more elegant than using None values as leaf nodes.
AaTreeSet.Node.EMPTY_LEAF = AaTreeSet.Node()


## What we need to know about AA trees

  - Nodes contain a numerical field named value.
  - AA trees are binary trees, therefore, each node has two (possibly empty) descendants, named left and right.
  - All values bigger than that of any node are stored in the subtree node.right and all the smaller ones in node.left
  - Each insertion or deletion to the tree "magically" balances the tree. (We are not yet interested in how that works)


## The problem we are presenting

  - We are searching in massive data, for instance all possible models that explain a phenomenon in a virtually unlimited space.
  - We only have a "quick and dirty" score of how good the model could be (e.g., weighted correlation average of its factors).
  - Fitting the model is computationally expensive, we want to fit the best models.
  - When we fit good models, other candidate models that are variations of the fitted ones are interesting candidates to evaluate and join the priority queue.
  
So we have a priority queue from which any model gets in and only the best candidate gets out. In practice, searches like these take place in fields like human genetics, run for many days, evaluate millions of models and discard billions of lower scored models. The queue is even limited in size because a model waiting in the billionth position has no chance of making the top.

## Limitations we assume

  - This implementation is purely for educational purposes and does not support storing anything (e.g. the model) in the nodes.
  - It does not (at least transparently) handle identical values in different nodes.
  
So we will only work with node.value and all values will be different.

## Task 1

  - Write a function that returns an aa tree filled with a small set of different values in different orders based on some parameter.

In [2]:
def order(seed):
    nn = list(range(20))
    mm = list()
    while len(nn) > 0:
        i = seed % len(nn)
        mm.append(nn[i])
        nn.remove(nn[i])
    return mm

print(order(5))
print(order(9))

def aa_tree(seed):
    nn   = order(seed)
    tree = AaTreeSet()
    for n in nn:
        tree.add(n)
    return tree


[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0, 2, 4, 3, 1]
[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0, 2, 4, 6, 8, 3, 1, 7, 5]


## Task 2

  - Write a function that returns the highest value (top priority) in a tree and removes its node from the tree.

In [3]:
def pop_highest(tree):
    node = tree.root
    val  = None
    while node is not AaTreeSet.Node.EMPTY_LEAF:
        val  = node.value
        node = node.right

    if val is not None:
        tree.remove(val)

    return (val)


## Task 3

  - Write a test that verifies that your function works properly for the same data given in different orders.

In [4]:
tree_1 = aa_tree(5)
tree_2 = aa_tree(9)

while True:
    n1 = pop_highest(tree_1)
    if n1 is None:
        break
    n2 = pop_highest(tree_2)
    if n1 != n2:
        raise(AssertionError)

print('Test passed.')

Test passed.
